<a href="https://colab.research.google.com/github/reidnclark/MFDPRepo/blob/main/mean_reversion_trading_algorithm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

side notes
- Term-End Poster
- Instead of attempting to adjust variables in the traditional DCF model due to insider info missing, I will try to address over 50% of the discrepancy in stock prices by analyzing the price history

side notes
- AI DJ:
-- if an audience member requests a song, along with an offer to pay for the song to occur...
-- need to have a live, adaptive neural network that measures risk and reward throughout your perfomrances. It is initially setup with variables at the beginnning too, though.

# **Multipurpose Financial Data Processor (MFDP)**
---

### Contents Summary:

|Analysis Section|Values|
|-|-|
|**1. Company Overview**|- Competitive Position|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**2. Financial Statements Analysis**|- **Income Statement**|
||----- Revenue Growth Rate|
||----- Profit Margins (Gross, Operating, Net)|
||----- Earnings Per Share (EPS) & EPS Growth|
||- **Balance Sheet**|
||----- Debt-to-Equity Ratio|
||- **Cash Flow Statement**|
||----- Cash Flow from Operating Activities|
||----- Cash Flow from Investing and Financing Activities|
||----- Free Cash Flow Analysis|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**3. Key Financial Ratios**|- **Profitability Ratios**|
||----- Return-on-Equity (ROE)|
||----- Return-on-Assets (ROA)|
||----- Net Profit Margin|
||- **Liquidity Ratios**|
||----- Current Ratio|
||----- Quick Ratio|
||- **Efficiency Ratios**|
||----- Asset Turnover Ratio|
||----- Inventory Turnover Ratio|
||- **Leverage Ratios**|
||----- Debt-to-Equity Ratio|
||----- Interest Coverage Ratio|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**3. Valuation**|- Price-to-Earnings|
||- Price-to-Book|
||- Discounted Cash Flow (DCF) Analysis|
||- **Other Valuation Models**|
||----- Dividend Discount Model (DDM)|
||----- EBITDA Multiples|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**4. Qualitative Analysis**|- Price-to-Earnings|
||- Price-to-Book|
||- Discounted Cash Flow (DCF) Analysis|
||- **Other Valuation Models**|
||----- Dividend Discount Model (DDM)|
||----- EBITDA Multiples|

---
# **1) Fundamental Analysis**

#### Import Libraries:

In [ ]:
import yfinance as yf
import pandas as pd
from babel.numbers import format_currency, format_percent

#### Get Data Function (Fundamental Analysis, All):

In [ ]:
def get_data(portfolio: list[str],
             basic_attributes: list[str],
             basic_monetary_attributes: list[str],
             basic_percentage_attributes: list[str],
             income_stmt_attributes: list[str],
             bal_sheet_attributes: list[str],
             cash_flow_attributes: list[str]) -> pd.DataFrame:


    combined_data = {}

    for ticker_symbol in portfolio:

        ticker = yf.Ticker(ticker_symbol)

        company_name = ticker.info.get('longName', ticker_symbol)

        basic_info = ticker.info
        basic_info_data = {attr: basic_info[attr] for attr in basic_attributes}

        basic_info_monetary_data = {
            attr: format_currency(basic_info[attr], 'USD', locale='en_US')
            for attr in basic_monetary_attributes
        }

        basic_info_percentage_data = {
            attr: format_percent(basic_info[attr], format="#,##0.00%", locale='en_US')
            for attr in basic_percentage_attributes
        }

        income_stmt = ticker.quarterly_income_stmt
        income_stmt_data = {
            attr: format_currency(income_stmt.loc[attr].iloc[0], 'USD', locale='en_US')
            for attr in income_stmt_attributes
        }

        bal_sheet = ticker.quarterly_balance_sheet
        bal_sheet_data = {
            attr: format_currency(bal_sheet.loc[attr].iloc[0], 'USD', locale='en_US')
            for attr in bal_sheet_attributes
        }

        cash_flow = ticker.quarterly_cashflow
        cash_flow_data = {
            attr: format_currency(cash_flow.loc[attr].iloc[0], 'USD', locale='en_US')
            for attr in cash_flow_attributes
        }

        # Organize df here
        combined_data[f"{company_name} ({ticker_symbol})"] = {
            **basic_info_monetary_data,
            **income_stmt_data,
            **basic_info_percentage_data,
            **basic_info_data,
            **bal_sheet_data, # nothin here yet, bal_sheet inputs are empty
            **cash_flow_data,
        }


    combined_df = pd.DataFrame.from_dict(combined_data, orient='index')

    return combined_df

---
#### **Input Arguments**:

In [ ]:
portfolio = ['GOLD', 'KGC', 'AEM', 'WPM']

# shorthand legend:
# SE = Shareholders' Equity
# TTM = Trailing Twelve Months

basic_attributes = ['trailingPE',
                    'forwardPE', # this is based on something from yfinance, possibly analyst consensus.
                    # in a future version replace forward PE with a manual analysis.
                    'priceToBook', # how much investors are willing to pay for each dollar worth of assets.
                    'debtToEquity', # (Total Debt / Total Equity)
                    'currentRatio', # ability to pay off short-term liabilities, considering all assets.
                    'quickRatio', # ability to pay off short-term liabilites, considering most-liquid assets.
                    #'interestCoverage'
                    #'assetTurnover', # efficiency ratio of converting assets to generate sales revenue.
                    #'inventoryTurnover' # efficiency ratio of converting inventory into sales, for 12-mo period.
                    # may have to calc both turnovers manually using manual bal sheet data
                    ]

# basic_monetary_attributes is for basic attributes that you want formatted as currency.
basic_monetary_attributes = ['marketCap',
                             'trailingEps',
                             ]

# ditto but if you want format it as percent.
basic_percentage_attributes = ['profitMargins', # growth rate of previous 12-months (%).
                               'returnOnEquity', # shareholders' return, per every dollar of profit. (Net Income / SE)
                               'returnOnAssets', # profit earned per every dollar of assets. (Net Income / Total Assets)
                               ]

income_stmt_attributes = ['Basic EPS',
                          'Gross Profit',
                          'Net Income']

bal_sheet_attributes = []

cash_flow_attributes = ['Free Cash Flow',
                        'Operating Cash Flow',
                        'Financing Cash Flow',
                        'Investing Cash Flow',
                        ]

---
### Access Results
---

In [ ]:
combined_data_df = get_data(portfolio,
                            basic_attributes,
                            basic_monetary_attributes,
                            basic_percentage_attributes,
                            income_stmt_attributes,
                            bal_sheet_attributes,
                            cash_flow_attributes,
                            )

#combined_data_df.sort_values(by=['marketCap'], ascending=False, inplace=True)
combined_data_df.sort_values(by=['marketCap'], ascending=False, inplace=True)
# uses inplace=True to directly modify the df, rather than assigning it back to a variable.

### **Print Results**:
---

In [ ]:
combined_data_df

,marketCap,trailingEps,Basic EPS,Gross Profit,Net Income,profitMargins,returnOnEquity,returnOnAssets,trailingPE,forwardPE,priceToBook,debtToEquity,currentRatio,quickRatio,Free Cash Flow,Operating Cash Flow,Financing Cash Flow,Investing Cash Flow
Agnico Eagle Mines Limited (AEM),"$39,958,581,248.00",$1.16,$0.95,"$926,248,000.00","$472,016,000.00",8.49%,3.10%,3.80%,68.465515,18.643190,1.985401,9.941,1.514,0.619,"$554,063,000.00","$961,336,000.00","-$137,234,000.00","-$424,576,000.00"
Barrick Gold Corporation (GOLD),"$35,326,611,456.00",$0.86,$0.21,"$1,183,000,000.00","$370,000,000.00",12.78%,7.20%,4.00%,23.337208,11.275281,1.489535,14.538,3.011,1.924,"$340,000,000.00","$1,159,000,000.00","-$362,000,000.00","-$703,000,000.00"
Wheaton Precious Metals Corp. (WPM),"$27,788,126,208.00",$1.26,$0.27,"$186,192,000.00","$122,317,000.00",50.44%,8.16%,5.35%,48.380950,34.834286,3.862384,0.080,26.858,26.647,"$188,710,000.00","$234,393,000.00","-$131,848,000.00","$131,693,000.00"
Kinross Gold Corporation (KGC),"$12,178,146,304.00",$0.40,$0.17,"$387,600,000.00","$210,900,000.00",10.91%,7.81%,5.16%,24.625000,13.133334,1.915597,31.889,1.271,0.426,"$329,800,000.00","$604,000,000.00","-$228,500,000.00","-$302,300,000.00"


# Long Stocks:
- ### Siemens Energy AG (ENR)
- ### Nutrien (NTR)
- ### Shin-Etsu Chemical Co. Ltd. (SHECY)
- ### Taiwan Semiconductor Mfg. Co. Ltd. (TSM)

---
# **2) DCF Model**

In [961]:
ticker_input = 'ENR'

financial_attributes_input = ['EBITDA'] # just use one for forecaster

cashflow_attributes_input = ['Free Cash Flow'] # ditto above

balsheet_attributes_input = ['Total Debt']

basic_attributes_input = ['beta', 'marketCap', 'sharesOutstanding']

#### Get Data Function:

In [962]:
def get_basic_data(attributes_data, attributes_input):
    basic_data_dict = {}
    for attr in attributes_input:
      basic_data_dict[attr] = attributes_data.get(attr)
    return basic_data_dict

def get_attr_hist_data(attributes_data, attributes_input):
    for attr in attributes_input:
      attr_hist_data = attributes_data.loc[attr].dropna()
    return attr_hist_data

def forecast(attr_hist_data):
    initial_value = attr_hist_data.iloc[-1]
    ending_value = attr_hist_data.iloc[0]
    #compound annual growth rate
    cagr = ((ending_value / initial_value)**(1/len(attr_hist_data))) - 1
    print(f'cagr of {attr_hist_data.name}: {cagr*100}%')
    forecasted_list = []
    for i in range(len(attr_hist_data)):
      forecast_value = ending_value * ((1+cagr)**(i+1))
      forecasted_list.append(forecast_value)
    return forecasted_list

def get_data(ticker_input: tuple,
             financial_attributes_input: list[str],
             cashflow_attributes_input: list[str],
             balsheet_attributes_input: list[str],
             basic_attributes_input: list[str]):

  yf_ticker = yf.Ticker(ticker_input)

  financial_attributes = yf_ticker.financials
  financial_attributes_hist_data = get_attr_hist_data(financial_attributes, financial_attributes_input)

  forecasted_list_financials = forecast(financial_attributes_hist_data)

  cashflow_attributes = yf_ticker.cashflow
  cashflow_attributes_hist_data = get_attr_hist_data(cashflow_attributes, cashflow_attributes_input)
  forecasted_list_cashflow = forecast(cashflow_attributes_hist_data)

  balsheet_attributes = yf_ticker.balance_sheet
  balsheet_attributes_hist_data = get_attr_hist_data(balsheet_attributes, balsheet_attributes_input).iloc[0]

  basic_attributes = yf_ticker.info
  basic_attributes_hist_data = get_basic_data(basic_attributes, basic_attributes_input)

  return forecasted_list_financials, forecasted_list_cashflow, balsheet_attributes_hist_data, basic_attributes_hist_data, yf_ticker






forecasted_list_financials, forecasted_list_cashflow, balsheet_attributes_hist_data, basic_attributes_hist_data, yf_ticker = get_data(ticker_input,
                                                                           financial_attributes_input,
                                                                           cashflow_attributes_input,
                                                                           balsheet_attributes_input,
                                                                           basic_attributes_input
                                                                                      )

#print(forecasted_list_financials)
#print(forecasted_list_cashflow)
#print(balsheet_attributes_hist_data)
#print(basic_attributes_hist_data)

cagr of EBITDA: 5.672145113227622%
cagr of Free Cash Flow: 2.125121402702246%


#### Get Beta, Total Equity, Total Debt, Cost of Debt, Cash, Shares Outstanding:

In [963]:
interest_expense = (yf_ticker.income_stmt.loc['Interest Expense'].iloc[0])

beta = basic_attributes_hist_data['beta']
total_equity = basic_attributes_hist_data['marketCap']
total_shares = basic_attributes_hist_data['sharesOutstanding']

total_debt = (yf_ticker.balance_sheet.loc['Total Debt'].iloc[0])
cost_of_debt = interest_expense / total_debt
total_cash = (yf_ticker.balance_sheet.loc['Cash And Cash Equivalents'].iloc[0])

#### **CAPM** Formula Function:
*Also known as "Capital Asset Pricing Model"*

---

In [964]:
r_f = 0.0475 # risk-free rate ( return subscript f, % )
b_i = beta # beta ( beta subscript i, ration )
e__r_m = 0.10 # expected market return ( e(return subscript m, %) )

e__r_i = r_f + b_i*(e__r_m - r_f) # expected return on investment
e__r_i

0.1017325

#### **WACC** Formula Function:
*Also known as "Weighted Average Cost of Capital"*

---

In [965]:
E = total_equity
D = total_debt
R_e = e__r_i # capm result
R_d = cost_of_debt

T = 0.25 # tax rate

wacc = (E / (E+D))*R_e + (D / (E+D))*R_d*(1-T) # weighted average cost of capital
wacc

0.06215084524934689

#### **Terminal Value (TV)** Formula Function:

---

In [966]:
fcf_n = forecasted_list_cashflow[-1] # value of final year in fcf forecast
g = 0.03 # perpetual growth rate (can use GDP growth estimate per year, (3 %), or ind.)

tv = (fcf_n * (1+g)) / (wacc - g)

def format_dollar(amount):
    """Format an integer as a dollar amount."""
    return f"${amount:,.2f}"

format_dollar(tv)
# in later version, add in tv financial metric multiple to find average. for now can just use perp.

'$11,792,490,288.69'

#### **Discount** Function and Ent, Eq:

---

In [967]:
pv_of_fcfs = []
discount_factors = []
for i in range(len(forecasted_list_cashflow)):
  discount_factor = (1 / ((1+wacc)**(i+1)))
  discount_factors.append(discount_factor)
  fcf_at_i = discount_factor * forecasted_list_cashflow[i]
  pv_of_fcfs.append(fcf_at_i)

pv_of_tv = discount_factors[-1]*tv

enterprise_value = (sum(pv_of_fcfs)) + pv_of_tv

equity_value = enterprise_value - total_debt + total_cash

share_price = equity_value / total_shares
#share_price
#pv_of_fcfs
#tv
#pv_of_tv

#### **Resulting Share Price**:

---

In [968]:
print(f'Intrinsic Price: ${share_price}')
print('------------------------------------')
current_price = yf_ticker.history(period='1d')['Close'].iloc[0]
print(f'Current Price: ${round((current_price),2)}')

Intrinsic Price: $101.15093963626387
------------------------------------
Current Price: $31.03



# IGNORE ALL BELOW
---
---
---

In [ ]:
import yfinance as yf
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import logging
# Suppress unnecessary logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Initialize sentiment analyzers
analyzer = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis")

# Define the ticker
ticker_symbol = 'FCX'  # Oracle
ticker = yf.Ticker(ticker_symbol)

# Define attributes
attributes = ['trailingEps']
income_stmt_attributes = ['Gross Profit', 'Net Income', 'Basic EPS']

def get_data(ticker: tuple, attributes: list[str]):
  info = ticker.info
  return {attr: info.get(attr) for attr in attributes}

def get_income_statement(ticker: tuple, income_stmt_attributes: list[str]):
    income_stmt = ticker.quarterly_income_stmt
    return {attr: income_stmt.loc[attr].values[0] for attr in income_stmt_attributes if attr in income_stmt.index}

def get_balance_sheet(ticker: tuple, bal_sheet_attributes):
  bal_sheet = ticker.quarterly_balance_sheet
  return {attr: bal_sheet.loc[attr].values[0] for attr in bal_sheet_attributes if attr in bal_sheet.index}

def get_combined_sentiment(ticker):
    news = ticker.news
    total_combined_score = 0
    article_count = 0

    for article in news:

        title = article['title']
        print(title)

        # VADER analysis
        vader_score = analyzer.polarity_scores(title)['compound']
        print(vader_score)

        # Hugging Face analysis
        context_article = f"Investing in {ticker.info['longName']}, {title}"
        hf_result = sentiment_pipeline(context_article)[0]
        hf_score = hf_result['score'] if hf_result['label'] == 'POSITIVE' else -hf_result['score']
        print(hf_score)

        # Weighted average of the scores
        combined_score = (0.6 * vader_score) + (0.4 * hf_score)

        # Update cumulative score and count
        total_combined_score += combined_score
        article_count += 1

        print(total_combined )

    # Calculate average sentiment score
    return (total_combined_score / article_count) if (article_count > 0) else None

# Get data and sentiment
data_values = get_data(ticker, attributes)
income_values = get_income_statement(ticker, income_stmt_attributes)
cumulative_sentiment = get_combined_sentiment(ticker)

# Combine all values into a single dictionary
data_values.update(income_values)
data_values['Cumulative Sentiment'] = cumulative_sentiment

# Create DataFrame
df_attributes = pd.DataFrame(list(data_values.items()), columns=['Attribute', 'Value'])

# Display the combined DataFrame
#print(df_attributes)
df_attributes


ModuleNotFoundError: No module named 'vaderSentiment'

In [ ]:
import yfinance as yf
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from transformers import pipeline
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
import yfinance as yf
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import logging

# Suppress unnecessary logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Initialize sentiment analyzers
analyzer = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis")

# Define the ticker
ticker_symbol = 'MSFT'  # Oracle
ticker = yf.Ticker(ticker_symbol)

# Define attributes
attributes = ['trailingEps']
income_stmt_attributes = ['Gross Profit', 'Net Income', 'Net Income Continuous Operations', 'Basic EPS']

def get_data(ticker, attributes):
    info = ticker.info
    return {attr: info.get(attr) for attr in attributes}

def get_income_statement(ticker, income_stmt_attributes):
    income_stmt = ticker.quarterly_income_stmt
    return {attr: income_stmt.loc[attr].values[0] for attr in income_stmt_attributes if attr in income_stmt.index}

def get_combined_sentiment(ticker):
    news = ticker.news
    total_combined_score = 0
    article_count = 0

    for article in news:
        title = article['title']

        # VADER analysis
        vader_score = analyzer.polarity_scores(title)['compound']

        # Hugging Face analysis
        context_article = f"Investing in {ticker.info['longName']}, {title}"
        hf_result = sentiment_pipeline(context_article)[0]
        hf_score = hf_result['score'] if hf_result['label'] == 'POSITIVE' else -hf_result['score']

        # Weighted average of the scores
        combined_score = (0.6 * vader_score) + (0.4 * hf_score)

        # Update cumulative score and count
        total_combined_score += combined_score
        article_count += 1

    return total_combined_score / article_count if article_count > 0 else None

# Get data and sentiment
data_values = get_data(ticker, attributes)
income_values = get_income_statement(ticker, income_stmt_attributes)
cumulative_sentiment = get_combined_sentiment(ticker)

# Combine all values into a single dictionary
data_values.update(income_values)
data_values['Cumulative Sentiment'] = cumulative_sentiment

# Create DataFrame
df_attributes = pd.DataFrame(list(data_values.items()), columns=['Attribute', 'Value'])

# Display the combined DataFrame
# print(df_attributes)
df_attributes


In [ ]:
import yfinance as yf
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import logging

In [ ]:
# Suppress unnecessary logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Initialize sentiment analyzers
analyzer = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
ticker_symbol = 'AAPL'  # Oracle
ticker = yf.Ticker(ticker_symbol)

basic_attributes = ['trailingEps']

income_stmt_attributes = ['Gross Profit', 'Net Income', 'Net Income Continuous Operations', 'Basic EPS']

In [ ]:
def get_basic_data(ticker: tuple, basic_attributes: list[str]):

    info = ticker.info

    return {info.get(i) for i in basic_attributes}


def get_income_stmt_data(ticker: tuple, income_stmt_attributes: list[str]) -> pd.DataFrame:

  income_stmt_data = ticker.quarterly_income_stmt
  # income_stmt_data_indexes = income_stmt_data.index

  # get metrics from income statement

  #return income_stmt_data
  print(income_stmt_attributes[0])
  return income_stmt_data


def get_news_sentiment(ticker):

    recent_news = ticker.news

    article_count = 0
    overall_avg_sentiment = 0

    for article in recent_news:

        title = article['title']

        # VADER analysis. 'compound' means value between -1 and 1.
        vader_score = analyzer.polarity_scores(title)['compound']

        # Hugging Face analysis. context_article gives Hugging Face context for scoring a value.
        context_article = f"Does this text support or talk badly of either {ticker.info['longName']} or \
                           {ticker.info['industry']} in the text {title}"
        hf_result = sentiment_pipeline(context_article)[0]
        hf_score = hf_result['score'] if hf_result['label'] == 'POSITIVE' else -hf_result['score']

        # Weighted average of the scores
        combined_score = (0.6 * vader_score) + (0.4 * hf_score)

        # Update cumulative score and count
        overall_avg_sentiment += combined_score
        article_count += 1

    return overall_avg_sentiment / article_count if article_count > 0 else None

# Get data and sentiment
basic_data = get_data(ticker, attributes)

income_stmt_data = get_income_stmt_data(ticker, income_stmt_attributes)

overall_avg_sentiment = get_combined_sentiment(ticker)


results = basic_data, overall_avg_sentiment, income_stmt_data

print(results)

# Create DataFrame
# data_values['Cumulative Sentiment'] = cumulative_sentiment
# df_attributes = pd.DataFrame(list(data_values.items()), columns=['Attribute', 'Value'])

# Display the combined DataFrame
#print(df_attributes)


In [ ]:
e# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
# Load sentiment-analysis pipeline from Hugging Face
sentiment_pipeline = pipeline("sentiment-analysis")

ticker = 'ORA' # Oracle
ticker = yf.Ticker(ticker)
attributes = ['trailingEps']
period = ['2023-01-01', '2024-01-01']

income_stmt = ticker.quarterly_income_stmt
income_stmt_attributes = ['Gross Profit', 'Net Income', 'Net Income Continuous Operations',
                          'Basic EPS']
# income_stmt_headers = (ticker.quarterly_income_stmt).index
# Optional: 'Diluted EPS', 'Net Income From Continuing And Discontinued Operation'
# print(income_stmt.index)

print(income_stmt_attributes)

def get_data(ticker, attributes):
    info = ticker.info
    attr_values = [info.get(i) for i in attributes]
    df_attributes = pd.DataFrame({'Attribute': attributes, 'Value': attr_values})
    return df_attributes

def get_combined_sentiment(ticker):
    news = ticker.news
    total_combined_score = 0
    article_count = 0

    for i in news:
        article = i['title']

        # VADER analysis
        vader_sentiment_dict = analyzer.polarity_scores(article)
        vader_score = vader_sentiment_dict['compound']

        # Hugging Face analysis
        context_article = f"Investing in {ticker.info['longName']}, {article}"
        hf_result = sentiment_pipeline(context_article)[0]
        hf_score = hf_result['score'] if hf_result['label'] == 'POSITIVE' else -hf_result['score']

        # Weighted average of the scores
        combined_score = (0.6 * vader_score) + (0.4 * hf_score)

        # Update cumulative score and count
        total_combined_score += combined_score
        article_count += 1

    # Calculate cumulative sentiment score
    if article_count > 0:
        cumulative_sentiment_score = total_combined_score / article_count
        #print(f"Cumulative Sentiment Score for {ticker.info['longName']}: {cumulative_sentiment_score:.2f}")
        # Return the cumulative sentiment score
        return cumulative_sentiment_score
    else:
        print("No articles to analyze.")
        return None

df_attributes = get_data(ticker, attributes)

# Get the cumulative sentiment score
cumulative_sentiment = get_combined_sentiment(ticker)

# Add cumulative sentiment as the last row
if cumulative_sentiment is not None:
    sentiment_row = pd.DataFrame({'Attribute': ['Cumulative Sentiment'], 'Value': [cumulative_sentiment]})
    # Concatenate the original DataFrame and the new row
    df_attributes = pd.concat([df_attributes, sentiment_row], ignore_index=True)

# Display the combined DataFrame
df_attributes

In [ ]:
import yfinance as yf
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from transformers import pipeline
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Load sentiment-analysis pipeline from Hugging Face
sentiment_pipeline = pipeline("sentiment-analysis")

ticker = 'ORA'
ticker = yf.Ticker(ticker)
attributes = ['trailingEps', 'profitMargins', 'revenueGrowth']
period = ('2023-01-01', '2024-01-01')

def get_data(ticker, attributes):
    info = ticker.info
    attr_values = [info.get(i) for i in attributes]
    df_attributes = pd.DataFrame({'Attribute': attributes, 'Value': attr_values})
    return df_attributes

def get_combined_sentiment(ticker):
    news = ticker.news
    total_combined_score = 0
    article_count = 0

    for i in news:
        article = i['title']

        # VADER analysis
        vader_sentiment_dict = analyzer.polarity_scores(article)
        vader_score = vader_sentiment_dict['compound']

        # Hugging Face analysis
        context_article = f"Investing in {ticker.info['longName']}, {article}"
        hf_result = sentiment_pipeline(context_article)[0]
        hf_score = hf_result['score'] if hf_result['label'] == 'POSITIVE' else -hf_result['score']

        # Weighted average of the scores
        combined_score = (0.6 * vader_score) + (0.4 * hf_score)

        # Update cumulative score and count
        total_combined_score += combined_score
        article_count += 1

    # Calculate cumulative sentiment score
    if article_count > 0:
        cumulative_sentiment_score = total_combined_score / article_count
        #print(f"Cumulative Sentiment Score for {ticker.info['longName']}: {cumulative_sentiment_score:.2f}")
        # Return the cumulative sentiment score
        return cumulative_sentiment_score
    else:
        print("No articles to analyze.")
        return None

df_attributes = get_data(ticker, attributes)

# Get the cumulative sentiment score
cumulative_sentiment = get_combined_sentiment(ticker)

# Add cumulative sentiment as the third row
if cumulative_sentiment is not None:
    new_row = pd.DataFrame({'Attribute': ['Cumulative Sentiment'], 'Value': [cumulative_sentiment]})
    # Use -1 to select the last row instead of the undefined variable 'end'
    df_attributes = pd.concat([df_attributes.iloc[:2], new_row, df_attributes.iloc[-1:]]).reset_index(drop=True)

# Display the combined DataFrame
print(df_attributes)

Fundamental Analysis

In [ ]:
import yfinance as yf
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
ticker = 'NVDA'
ticker = yf.Ticker(ticker)
attributes = ['trailingEps', 'profitMargins', 'revenueGrowth']

def get_data(ticker: tuple, attributes: list[str], period: tuple[str, str]=None):

  info = ticker.info
  attr_values = [info.get(i) for i in attributes]
  df_attributes = pd.DataFrame({'Attribute': attributes, 'Value': attr_values})

  return df_attributes

df_attributes = get_data(ticker, attributes)
df_attributes


def get_sentiment(ticker: tuple):

  news = ticker.news

  for i in news:

    article = i['title']
    sentiment_dict = analyzer.polarity_scores(article)
    sentiment_score = sentiment_dict['compound']

    print(f"Title: {article}, Sentiment Score: {sentiment_score}")

sentiment = get_sentiment(ticker)


In [ ]:
from transformers import pipeline
import yfinance as yf
import pandas as pd

# Load a sentiment-analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

ticker = 'NVDA'
ticker = yf.Ticker(ticker)
attributes = ['trailingEps', 'profitMargins', 'revenueGrowth']

def get_data(ticker: tuple, attributes: list[str], period: tuple[str, str]=None):
    info = ticker.info
    attr_values = [info.get(i) for i in attributes]
    df_attributes = pd.DataFrame({'Attribute': attributes, 'Value': attr_values})
    return df_attributes

df_attributes = get_data(ticker, attributes)

def get_sentiment(ticker: tuple):
    news = ticker.news
    for i in news:
        article = i['title']
        result = sentiment_pipeline(article)[0]
        sentiment_label = result['label']  # 'POSITIVE' or 'NEGATIVE'
        sentiment_score = result['score']  # Confidence score
        print(f"Title: {article}, Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

sentiment = get_sentiment(ticker)

In [ ]:
import yfinance as yf
import pandas as pd
from transformers import pipeline

# Load the sentiment-analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

ticker = 'NVDA'
ticker_data = yf.Ticker(ticker)
attributes = ['trailingEps', 'profitMargins', 'revenueGrowth']

def get_data(ticker, attributes):
    info = ticker.info
    attr_values = [info.get(i) for i in attributes]
    df_attributes = pd.DataFrame({'Attribute': attributes, 'Value': attr_values})
    return df_attributes

df_attributes = get_data(ticker_data, attributes)

def get_sentiment(ticker):
    news = ticker.news
    for i in news:
        article = i['title']
        # Prepend specific context for stock analysis
        context_article = f"Relevance to {ticker} retail investors, {article}"
        result = sentiment_pipeline(context_article)[0]
        sentiment_label = result['label']
        sentiment_score = result['score']
        print(f"Title: {article}, Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

get_sentiment(ticker_data)


## Mean Reversion Trading Algorithm
---

# Full Code:

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as formatter
import pandas as pd
import datetime
pd.set_option('display.max_rows', None)


def get_data(tickers_list: list[str], start_date: str, end_date: str) -> pd.DataFrame:
  data = yf.download(tickers_list, start=start_date, end=end_date, progress=False)
  adj_close = pd.DataFrame(data['Adj Close'])
  adj_close.index = adj_close.index.date # Voids time signature on Date index, just Date
  return adj_close


def get_moving_avg(price_data: pd.DataFrame, shorter_moving_avg_interval: int,
                   longer_moving_avg_interval: int) -> tuple:
  moving_avg_shorter = price_data.rolling(window=shorter_moving_avg_interval).mean()
  moving_avg_longer = price_data.rolling(window=longer_moving_avg_interval).mean()
  moving_avgs = pd.DataFrame(index=price_data.index)
  for ticker in price_data.columns:
        moving_avgs[f'{ticker}_shorter'] = moving_avg_shorter[ticker]
        moving_avgs[f'{ticker}_longer'] = moving_avg_longer[ticker]
  return moving_avgs, moving_avg_shorter, moving_avg_longer


def get_sdev(price_data):
  sdev = price_data.std()
  return sdev


def plot_results(moving_averages: pd.DataFrame, original_close_prices: list[float],plot_colours: list[str], plot_line_widths: list[float],
                 plot_line_styles: list[str], tickers_list: list[str], plot_figsize: tuple, sdev: pd.Series()):
  plt.figure(figsize=plot_figsize, facecolor='white')
  plt.grid(True, linestyle='-', alpha=0.7)
  plt.gca().set_facecolor('lightgrey')
  plt.title((f'Historical Adjusted Close Prices & Moving Averages\n'
             f'for {tickers_list}'))
  plt.legend()
  plt.xlabel('Date', fontsize=13)
  plt.ylabel('Price Activity', fontsize=13)
  plt.gca().yaxis.set_major_formatter(formatter.StrMethodFormatter('${x:,.2f}'))
  for i, ticker in enumerate(moving_averages.columns):
    plt.plot(moving_averages.index, moving_averages[ticker], linewidth=plot_line_widths[i], linestyle=plot_line_styles[i],
             color=plot_colours[i], label=ticker)
  plt.plot(original_close_prices.index, original_close_prices, color='black',
           linewidth=0.9, label=f'{tickers_list[0]} Adj Close Price', alpha=0.7)
  plt.plot(moving_averages.index, moving_averages['Adj Close_longer'] + (sdev/2), linewidth=1.2, color='red', linestyle='--',
           alpha = 0.7)
  plt.plot(moving_averages.index, moving_averages['Adj Close_longer'] - (sdev/2), linewidth=1.2, color='darkgreen',
           linestyle='--', alpha = 0.7)
  plt.show()


plot_colours = ['blue', 'black'] # DEFAULT INDIVIDUAL
plot_line_widths = [1.75, 1] # DEFAULT INDIVIDUAL
plot_line_styles = ['-','-'] # DEFAULT INDIVIDUAL
plot_figsize = 9,6
tickers_list = ['FCX'] # DEFAULT INDIVIDUAL
#tickers_list = ['TSMC34.SA'] # DEFAULT INDIVIDUAL
start_date = '2021-01-01' # DEFAULT
end_date = '2024-01-01' # DEFAULT
shorter_rolling_interval = 20 # DEFAULT (NUMBER OF DAYS)
longer_rolling_interval = 40 # DEFAULT (NUMBER OF DAYS)


adj_close = get_data(tickers_list, start_date, end_date)
moving_avgs, moving_avg_shorter, moving_avg_longer = get_moving_avg(adj_close, shorter_rolling_interval, longer_rolling_interval)
sdev = (get_sdev(moving_avg_longer)).iloc[0]


plot_results(moving_avgs, adj_close, plot_colours, plot_line_widths, plot_line_styles, tickers_list, plot_figsize, sdev)

# Functions
---

Import Libraries:

In [ ]:
import yfinance as yf

import matplotlib.pyplot as plt
import matplotlib.ticker as formatter

import pandas as pd
pd.set_option('display.max_rows', None)

import datetime

**Function**: Get Data

In [ ]:
def get_data(tickers_list: list[str], start_date: str, end_date: str, interval=None):

  # Retrieve data for adjusted close prices
  data = yf.download(tickers_list, start=start_date, end=end_date, progress=False)
  hist =

  adj_close = pd.DataFrame(hist_data['Adj Close'])
  adj_close.index = adj_close.index.date # Voids time signature on Date index, just Date

  return hist_data, adj_close

In [ ]:
hist_data, adj_close = get_data(['MSFT'], '2023-06-01', '2024-01-01')

hist_data.info

**Function**: Get Moving Averages

In [ ]:
def get_moving_avg(price_data: pd.DataFrame, shorter_moving_avg_interval: int,
                   longer_moving_avg_interval: int) -> tuple:

  # Pandas rolling avg
  moving_avg_shorter = price_data.rolling(window=shorter_moving_avg_interval).mean()
  moving_avg_longer = price_data.rolling(window=longer_moving_avg_interval).mean()

  # Convert to DataFrame
  moving_avgs = pd.DataFrame(index=price_data.index)
  for ticker in price_data.columns:
        moving_avgs[f'{ticker}_shorter'] = moving_avg_shorter[ticker]
        moving_avgs[f'{ticker}_longer'] = moving_avg_longer[ticker]
  # Note: Replace Initialized (Combined) Columns with shorter and longer moving averages

  return moving_avgs, moving_avg_shorter, moving_avg_longer

**Function**: Get Sdev

In [ ]:
def get_sdev(price_data):

  sdev = price_data.std()

  return sdev

**Function**: Cycler

# Plot Results
---



In [ ]:
def trader(price_data: pd.DataFrame, sdev: int):

  sma_price = price_data['Adj Close_shorter']
  lma_price = price_data['Adj Close_longer']

  #for i in price_data:

    #if sma_price[i] and lma_price[i] == int:

      # begin trading

      #if (sma_price[i] - lma_price[i]) <= sdev:



    #else:
      #continue





**Function**:

In [ ]:
def plot_results(moving_averages: pd.DataFrame, original_close_prices: list[float],plot_colours: list[str], plot_line_widths: list[float],
                 plot_line_styles: list[str], tickers_list: list[str], plot_figsize: tuple, sdev: pd.Series()):


  plt.figure(figsize=plot_figsize, facecolor='white')
  plt.grid(True, linestyle='-', alpha=0.7)
  plt.gca().set_facecolor('lightgrey')
  # Bg color

  plt.title((f'Historical Adjusted Close Prices & Moving Averages\n'
             f'for {tickers_list}'))

  plt.legend()
  plt.xlabel('Date', fontsize=13)
  plt.ylabel('Price Activity', fontsize=13)
  plt.gca().yaxis.set_major_formatter(formatter.StrMethodFormatter('${x:,.2f}'))
  # Sets ($) data format

  for i, ticker in enumerate(moving_averages.columns):
    plt.plot(moving_averages.index, moving_averages[ticker], linewidth=plot_line_widths[i], linestyle=plot_line_styles[i],
             color=plot_colours[i], label=ticker)
  # Note: Loops through plot formatting in following code block (Input: Format Plot)

  # Note: Plot Original Price Data Always as Black w/ its own other style attributes
  # Change this terrible line later
  plt.plot(original_close_prices.index, original_close_prices, color='black',
           linewidth=0.9, label=f'{tickers_list[0]} Adj Close Price')

  # Plot sdev lines
  plt.plot(moving_averages.index, moving_averages['Adj Close_longer'] + sdev, linewidth=0.7)
  plt.plot(moving_averages.index, moving_averages['Adj Close_longer'] - sdev)


  plt.show()

# Inputs

---
**Inputs**: Format Plot

In [ ]:
#plot_colours = ['blue', 'blue', 'darkred', 'darkred'] # DEFAULT
plot_colours = ['blue', 'red'] # DEFAULT INDIVIDUAL
#plot_line_widths = [1,0.4,1,0.4] # DEFAULT
plot_line_widths = [1, 0.7] # DEFAULT INDIVIDUAL
#plot_line_styles = ['-','-.','-','-.'] # DEFAULT
plot_line_styles = ['-','--'] # DEFAULT INDIVIDUAL
plot_figsize = 9,6

**Inputs**: Portfolio Tickers, Date Period, Moving Average Rolling Intervals

In [ ]:
#tickers_list = ['MSFT', 'AAPL'] # DEFAULT
tickers_list = ['MSFT'] # DEFAULT INDIVIDUAL
start_date = '2023-01-01' # DEFAULT
end_date = '2024-01-01' # DEFAULT
shorter_rolling_interval = 5 # DEFAULT (NUMBER OF DAYS)
longer_rolling_interval = 15 # DEFAULT (NUMBER OF DAYS)

Store Function Output

In [ ]:
## Function: Get Data
adj_close = get_data(tickers_list, start_date, end_date)

## Function: Get Moving Averages (uses normalized adj close result)
moving_avgs, moving_avg_shorter, moving_avg_longer = get_moving_avg(adj_close, shorter_rolling_interval, longer_rolling_interval)

## Function: Get Sdev
sdev = (get_sdev(moving_avg_longer)).iloc[0]

print(sdev)

# Output:
---

**Return Results:**

In [ ]:
#plot_results(adj_close, plot_colours, plot_line_widths, plot_line_styles, tickers_list)


#plot_results(moving_avgs, adj_close, plot_colours, plot_line_widths, plot_line_styles, tickers_list, plot_figsize, sdev)

# Analysis

---
**Function**: Get Rolling Sdev



In [ ]:
def get_sdev(price_data):

  sdev = price_data.std()

  return sdev

sdev = (get_sdev(adj_close)).iloc[0]

print(sdev)

---
**Function**: Check if Current Price +/2 sdev about the mean